In [1]:
import pandas as pd
import json
import numpy as np

# Prepariamo lavoro per calcolo evoluzione fuorisede

In [67]:
df = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/b270ef1a-c219-48b1-8399-b1458e225d39/download/14_iscrittixresidenzasedecorsogruppo.csv', delimiter=';')

decoder = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/eae4ee94-0797-41d2-b007-bc6dad3ef3e2/download/07_iscrittixresidenza.csv', delimiter=';')

with open('decoders.json','r') as file:
    decoders = json.load(file)

cod_prov = decoders[0]
prov_cod = decoders[1]

df['ResidenzaCOD'] = df['ResidenzaR'].apply(lambda x: prov_cod[x])
df['sede_ateneo']=df['SedeP'].apply(lambda x: cod_prov.get(str(x),'Sconosciuta'))

In [68]:
with open('cod_confini.json','r') as file:
    dec_conf = json.load(file)

In [69]:
df['AnnoA'] = df['AnnoA'].str.extract('^(\d+)/')[0].astype(int)

In [70]:
filtro = [] 
for i,value in df.iterrows():
    prov = value['ResidenzaCOD']
    confini = dec_conf.get(str(prov),[])
    is_confinante = value['SedeP'] in confini 
    filtro.append(is_confinante)

In [71]:
fuorisede = df[df['SedeP']!=df['ResidenzaCOD']]

In [72]:
fuorisede_real = df[list(map(lambda x: not x, filtro))]

In [74]:
fuorisede_real = fuorisede_real[fuorisede_real['SedeP']!=fuorisede_real['ResidenzaCOD']]

In [9]:
def proporzioni(prov):

    f = fuorisede_real.groupby(by=['sede_ateneo','AnnoA']).agg({
    'Isc':sum
    })
    
    t = df.groupby(by=['sede_ateneo','AnnoA']).agg({
    'Isc':sum
    })
    
    return f.loc[prov]['Isc'] #torna il numero di studenti real fuorisede per anno in una provincia
#/t.loc[prov]['Isc']*100

In [11]:
proporzioni('BOLOGNA')

AnnoA
2010    30649
2011    30676
2012    30722
2013    31160
2014    32180
2015    33088
2016    34235
2017    34981
2018    35714
2019    35639
2020    36885
2021    39479
Name: Isc, dtype: int64

# Lavori preparatori per importare dataset DISU

In [25]:
links = [f'http://dati.ustat.miur.it/dataset/{x}-diritto-allo-studio-universitario-dsu-regionale'\
        for x in range(2014,2022)]

In [26]:
def leggi_df(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    finale = soup.find(text=re.compile(r'Alloggi e mense',flags=re.I))
    finale = finale.parent['href']
    r = requests.get('http://dati.ustat.miur.it'+finale)
    soup = BeautifulSoup(r.content,'html.parser')
    n_url = list(filter(lambda x: 'csv' in x['href'],soup.find_all('a')))[0]['href']
    return pd.read_csv(n_url,delimiter=';',encoding='ISO-8859-1')

In [27]:
lista = []
for url in links:
    ndf = leggi_df(url) 
    lista.append(ndf)

In [28]:
disu = pd.concat(lista,ignore_index=True)

In [29]:
disu = disu.dropna(how='all')

In [30]:
disu['DATA'].unique()

array(['01/11/2014', '01/11/2015', '01/11/2016', '1/11/17', '01/11/2018',
       '01/11/2019', '01/11/2020', '01-nov-20', '01/11/2021'],
      dtype=object)

In [31]:
def utility(x):
    if len(x)==4:
        return int(x)
    elif len(x)==2:
        return int('20'+x)

In [32]:
disu['DATA']=disu['DATA'].str.extract(r'/(\d+$)')[0].fillna('20').apply(utility)

In [33]:
disu = disu[disu['CODICE_SERVIZIO']=='1']

In [34]:
disu_f = disu.groupby(by=['NOME_REGIONE','DATA']).agg({
    'NUMERO':sum
})

In [80]:
prov_reg = dict(zip(df['ResidenzaR'],df['ResidenzaP']))

In [81]:
df['regione_ateno'] = df['sede_ateneo'].apply(lambda x: prov_reg.get(x, np.nan))

In [82]:
fuorisede_real['regione_ateno'] = fuorisede_real['sede_ateneo'].apply(lambda x: prov_reg.get(x, np.nan))

# EMILIA ROMAGNA

In [38]:
n_posti = disu_f.loc['Emilia-Romagna']['NUMERO']

n_fuorisede = fuorisede_real.groupby(by=['regione_ateno','AnnoA']).agg({
    'Isc':sum
}).loc['EMILIA ROMAGNA']['Isc']

In [39]:
n_posti[2021]/n_posti[2014]*100

101.74136454467599

In [40]:
n_fuorisede[2021]/n_fuorisede[2014]*100

146.82137075013492

In [41]:
n_posti/n_fuorisede*100

2010         NaN
2011         NaN
2012         NaN
2013         NaN
2014    6.301493
2015    6.081222
2016    5.792406
2017    5.339266
2018    5.076032
2019    4.865432
2020    4.585192
2021    4.366684
dtype: float64

# LOMBARDIA

In [42]:
n_posti = disu_f.loc['Lombardia']['NUMERO']

n_fuorisede = fuorisede_real.groupby(by=['regione_ateno','AnnoA']).agg({
    'Isc':sum
}).loc['LOMBARDIA']['Isc']

In [43]:
n_posti/n_fuorisede*100

2010          NaN
2011          NaN
2012          NaN
2013          NaN
2014    11.629452
2015    12.492436
2016    12.448435
2017    11.851852
2018    11.934698
2019    11.362805
2020    10.670375
2021    10.178806
dtype: float64

In [44]:
n_posti[2021]/n_posti[2014]*100

115.23845007451565

In [45]:
n_fuorisede[2021]/n_fuorisede[2014]*100

131.66180421156014

# PIEMONTE

In [46]:
n_posti = disu_f.loc['Piemonte']['NUMERO']

n_fuorisede = fuorisede_real.groupby(by=['regione_ateno','AnnoA']).agg({
    'Isc':sum
}).loc['PIEMONTE']['Isc']

In [47]:
n_posti/n_fuorisede*100

2010         NaN
2011         NaN
2012         NaN
2013         NaN
2014    9.750585
2015    8.849079
2016    8.078380
2017    7.492056
2018    6.999565
2019    6.767167
2020    6.678518
2021    6.419445
dtype: float64

In [48]:
n_posti[2021]/n_posti[2014]*100

100.52867019113461

In [49]:
n_fuorisede[2021]/n_fuorisede[2014]*100

152.69439708156548

# LAZIO

In [50]:
n_posti = disu_f.loc['Lazio']['NUMERO']

n_fuorisede = fuorisede_real.groupby(by=['regione_ateno','AnnoA']).agg({
    'Isc':sum
}).loc['LAZIO']['Isc']

In [51]:
n_posti/n_fuorisede*100

2010         NaN
2011         NaN
2012         NaN
2013         NaN
2014    6.209190
2015    6.538715
2016    6.659076
2017    6.940995
2018    7.154213
2019    7.241262
2020    6.586594
2021    5.756468
dtype: float64

In [52]:
n_posti[2021]/n_posti[2014]*100

100.43324143363527

In [53]:
n_fuorisede[2021]/n_fuorisede[2014]*100

108.33190677655229

# ITALIA

In [54]:
n_posti = disu.groupby('DATA').agg({
    'NUMERO':sum
})['NUMERO']

In [55]:
n_fuorisede = fuorisede_real.groupby('AnnoA').agg({
    'Isc':sum
})['Isc']

In [56]:
n_posti/n_fuorisede*100

2010          NaN
2011          NaN
2012          NaN
2013          NaN
2014    11.727409
2015    12.231285
2016    11.833815
2017    11.352843
2018    11.449624
2019    11.205516
2020    10.599871
2021     9.865654
dtype: float64

# Creiamo csv con dati per ogni regione

In [90]:
disu_fuoris = {}
for i in set(list(map(lambda x: x[0],disu_f.index))):
    disu_fuoris[i] = i.upper()

In [89]:
fuorisede_real['regione_ateno'].unique()

array(['LOMBARDIA', 'PIEMONTE', 'SICILIA', 'ABRUZZO', 'TOSCANA',
       'EMILIA ROMAGNA', 'CALABRIA', 'PUGLIA', 'VENETO', 'LAZIO',
       'SARDEGNA', 'FRIULI VENEZIA GIULIA', 'MARCHE', 'CAMPANIA',
       'LIGURIA', nan, 'BASILICATA', 'UMBRIA',
       'Provincia autonoma di Bolzano', 'Provincia autonoma di TRENTO',
       'MOLISE', "VALLE D'AOSTA"], dtype=object)

In [92]:
disu_fuoris['Friuli-Venezia Giulia'] = 'FRIULI VENEZIA GIULIA'

In [94]:
del disu_fuoris['Trentino-Alto Adige']

In [96]:
disu_fuoris['Emilia-Romagna'] = 'EMILIA ROMAGNA'

In [100]:
disu_fuoris

{'Calabria': 'CALABRIA',
 'Sicilia': 'SICILIA',
 'Veneto': 'VENETO',
 'Friuli-Venezia Giulia': 'FRIULI VENEZIA GIULIA',
 'Molise': 'MOLISE',
 'Abruzzo': 'ABRUZZO',
 'Piemonte': 'PIEMONTE',
 'Lombardia': 'LOMBARDIA',
 'Lazio': 'LAZIO',
 'Campania': 'CAMPANIA',
 'Basilicata': 'BASILICATA',
 'Liguria': 'LIGURIA',
 'Umbria': 'UMBRIA',
 "Valle d'Aosta": "VALLE D'AOSTA",
 'Toscana': 'TOSCANA',
 'Marche': 'MARCHE',
 'Emilia-Romagna': 'EMILIA ROMAGNA',
 'Puglia': 'PUGLIA',
 'Sardegna': 'SARDEGNA'}

In [99]:
disu_regioni = pd.DataFrame()

In [114]:
for regione in disu_fuoris:
    n_posti = disu_f.loc[regione]['NUMERO']
    n_fuorisede = fuorisede_real.groupby(by=['regione_ateno','AnnoA']).agg({
    'Isc':sum
    }).loc[disu_fuoris[regione]]['Isc']
    serie = n_posti.combine(n_fuorisede,lambda x, y: {'posti':x,
                                         'fuorisede':y,
                                         'rapporto':x/y*100})
    disu_regioni[regione]=serie

In [131]:
disu_regioni.to_csv('residenze_per_fuorisede.csv')